<a href="https://colab.research.google.com/github/LakhiCharanMahato/Image_to_Emotion/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle
!pip install --upgrade kaggle
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 601 ~/.kaggle/kaggle.json
#https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p /content
!unzip imdb-dataset-of-50k-movie-reviews.zip
import pandas as pd
df=pd.read_csv('IMDB Dataset.csv')
#train dataset
train_reviews=df.review[:40000]
train_sentiments=df.sentiment[:40000]
#test dataset
test_reviews=df.review[40000:]
test_sentiments=df.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

 35% 9.00M/25.7M [00:00<00:00, 70.4MB/s]
100% 25.7M/25.7M [00:00<00:00, 102MB/s] 
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        
(40000,) (40000,)
(10000,) (10000,)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup

In [3]:
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
from string import punctuation
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    text=''.join([c for c in text if c not in punctuation])
    text=denoise_text(text)
    text=simple_stemmer(text)
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [4]:
%%time
import re
df.review = df.review.apply(lambda x: preprocess(x))

CPU times: user 4min 37s, sys: 831 ms, total: 4min 38s
Wall time: 4min 38s


In [5]:
from sklearn.model_selection import train_test_split
TRAIN_SIZE = 0.80
df_train, df_test = train_test_split(df, test_size=1-TRAIN_SIZE, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

TRAIN size: 40000
TEST size: 10000


In [6]:
%%time
documents = [_text.split() for _text in df_train.review] 

CPU times: user 452 ms, sys: 186 ms, total: 638 ms
Wall time: 638 ms


In [7]:
import gensim
# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)
w2v_model.build_vocab(documents)

In [8]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 17836


In [9]:
%%time
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

CPU times: user 15min 5s, sys: 3.9 s, total: 15min 8s
Wall time: 7min 45s


(148837349, 169839264)

In [10]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.review)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

Total words 64128
CPU times: user 5.14 s, sys: 44 ms, total: 5.18 s
Wall time: 5.21 s


In [11]:
from keras.preprocessing.sequence import pad_sequences
SEQUENCE_LENGTH = 300
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.review), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.review), maxlen=SEQUENCE_LENGTH)

In [12]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df_train.sentiment.tolist())

LabelEncoder()

In [13]:
y_train = encoder.transform(df_train.sentiment.tolist())
y_test = encoder.transform(df_test.sentiment.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)
import numpy as np
embedding_matrix = np.zeros((vocab_size, W2V_SIZE))

for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

y_train (40000, 1)
y_test (10000, 1)
(64128, 300)


In [14]:
from __future__ import absolute_import,division,print_function
import tensorflow as tf
import tensorflow_datasets as tfds
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import math
import numpy as np
import matplotlib.pyplot as plt

import tqdm
import tqdm.auto
tqdm.tqdm=tqdm.auto.tqdm
print(tf.__version__)
tf.compat.v1.enable_eager_execution()
#tf.debugging.set_log_device_placement(False)

2.5.0


In [15]:
xtrain=tf.constant(x_train)
ytrain=tf.constant(y_train)
xtest=tf.constant(x_test)
ytest=tf.constant(y_test)
max_words=300

In [16]:
model6=tf.keras.Sequential([
                           tf.keras.layers.Embedding(vocab_size,100,
                                                     #mask_zero=True
                                                     input_length=max_words
                                                     ),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.Bidirectional(
                               tf.keras.layers.GRU(128,#dropout=0.4,
                                               return_sequences=True)
                           ),                            
                           tf.keras.layers.Conv1D(32,kernel_size=3,
                                                  padding='same',
                                                  activation=tf.nn.relu),
                           tf.keras.layers.GlobalMaxPooling1D(),
                           tf.keras.layers.Dense(64,activation=tf.nn.relu),  
                           tf.keras.layers.Dropout(0.5),
                           tf.keras.layers.Dense(1,activation=tf.nn.sigmoid),
])

In [17]:
model6.compile(#optimizer='adam',
              tf.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model6.fit(xtrain,ytrain,
          validation_data=(xtest,ytest),
          epochs=3,
          batch_size=128,
          #train_dataset,
          verbose=1,
          #callbacks = [early_stop]
          )

Epoch 1/3
313/313 [==============================] - 676s 2s/step - loss: 0.3827 - accuracy: 0.8184 - val_loss: 0.2552 - val_accuracy: 0.8956
Epoch 2/3
313/313 [==============================] - 668s 2s/step - loss: 0.1884 - accuracy: 0.9316 - val_loss: 0.2463 - val_accuracy: 0.9020
Epoch 3/3
313/313 [==============================] - 647s 2s/step - loss: 0.1120 - accuracy: 0.9615 - val_loss: 0.2841 - val_accuracy: 0.8940


In [18]:
import time
def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    text=text.replace("\n", "")
    text=preprocess(text)
    if len(text)==0:
      return 300
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    #feature_tensor = torch.from_numpy(x_test)
    feature_tensor = tf.constant(x_test)
    pred=model6.predict_classes(feature_tensor,verbose=1)
    #pred = torch.round(output.squeeze()
    #return pred.item()
    return pred[0][0]

In [19]:
!unzip /content/9e9afdaabac711ea.zip

Archive:  /content/9e9afdaabac711ea.zip
   creating: Data Files/
   creating: Data Files/Dataset/
  inflating: Data Files/Dataset/Test903.jpg  
  inflating: Data Files/Dataset/Test2434.jpg  
  inflating: Data Files/Dataset/Test145.jpg  
  inflating: Data Files/Dataset/Test1615.jpg  
  inflating: Data Files/Dataset/Test1122.jpg  
  inflating: Data Files/Dataset/Test132.jpg  
  inflating: Data Files/Dataset/Test2959.jpg  
  inflating: Data Files/Dataset/Test109.jpg  
  inflating: Data Files/Dataset/Test114.jpg  
  inflating: Data Files/Dataset/Test230.jpg  
  inflating: Data Files/Dataset/Test536.jpg  
  inflating: Data Files/Dataset/Test516.jpg  
  inflating: Data Files/Dataset/Test233.jpg  
  inflating: Data Files/Dataset/Test2792.jpg  
  inflating: Data Files/Dataset/Test119.jpg  
  inflating: Data Files/Dataset/Test2616.jpg  
  inflating: Data Files/Dataset/Test1934.jpg  
  inflating: Data Files/Dataset/Test803.jpg  
  inflating: Data Files/Dataset/Test3030.jpg  
  inflating: Data Fi

In [20]:
!sudo apt install tesseract-ocr
!pip install pytesseract
import pytesseract
import cv2
from google.colab.patches import cv2_imshow
import pandas as pd
df1=pd.read_csv('Data Files/Test.csv')
m=df1['Filename'].tolist()
mh=[]
for i in m:
  c='Data Files/Dataset/'+i
  img=cv2.imread(c)
  #img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  aa=pytesseract.image_to_string(img)
  mh.append(aa)
  #cv2_imshow(img) 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (3,523 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [21]:
labe=['']*len(mh)
co=0
#rot=[300]*len(mh)
for i in range(len(mh)):
  if len(mh[i])==0:
    labe[i]='Random'
    co+=1
    continue
  blob=predict(mh[i])
  print(blob)

  #po=blob.sentiment.polarity
  if blob==300:
    labe[i]='Random'
    co+=1
    continue
  if blob==1:
    labe[i]='Positive'
  else:
    labe[i]='Negative'

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


1/1 [==============================] - 1s 662ms/step
1
1/1 [==============================] - 0s 34ms/step
1
1/1 [==============================] - 0s 37ms/step
1
1/1 [==============================] - 0s 46ms/step
1
1/1 [==============================] - 0s 40ms/step
0
300
1/1 [==============================] - 0s 36ms/step
0
1/1 [==============================] - 0s 44ms/step
1
1/1 [==============================] - 0s 40ms/step
1
1/1 [==============================] - 0s 39ms/step
1
300
300
300
300
1/1 [==============================] - 0s 40ms/step
0
1/1 [==============================] - 0s 40ms/step
1
300
1/1 [==============================] - 0s 35ms/step
1
1/1 [==============================] - 0s 35ms/step
0
1/1 [==============================] - 0s 39ms/step
1
1/1 [==============================] - 0s 42ms/step
0
300
1/1 [==============================] - 0s 34ms/step
1
1/1 [==============================] - 0s 38ms/step
1
300
300
300
1/1 [==============================] - 0s

In [22]:
df1['Category']=labe
df1.to_csv('result41.csv',index=False)